In [ ]:
%env MODEL_DIR=/content/Cats

# Install Dependencies

In [ ]:
%env DARKNET_VERSION=5.1.82
!if [[ ! -d /content/darknet_tflite ]]; then git clone --depth 1 --branch main https://github.com/sventschui/darknet-tflite.git /content/darknet_tflite; fi
!/content/darknet_tflite/scripts/install_dependencies.sh

In [ ]:
!cd /content/Cats && darknet detector calc_anchors /content/Cats/Cats.data -num_of_clusters 6 -width 384 -height 288


# Unpack dataset

In [ ]:
!/content/darknet_tflite/scripts/unpack_cats.sh

# Train model

In [ ]:
# Mount GDrive, only possible in cloud colab notebooks
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash

set -e

rm -rf /content/drive/MyDrive/nn/cats2
mkdir -p /content/drive/MyDrive/nn/cats2
cp "${MODEL_DIR}/Cats.cfg" "${MODEL_DIR}/Cats.data" /content/drive/MyDrive/nn/cats2

cd /content/drive/MyDrive/nn/cats2

darknet detector train \
  "${MODEL_DIR}/Cats.data" \
  "${MODEL_DIR}/Cats.cfg" \
  -map \
  -dont_show #\
  # Cat_best.weights \
  # -clear

cp "${MODEL_DIR}/*.weights" /content/drive/MyDrive/nn/cats2